In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pymongo
import pandas as pd
from bson import ObjectId
import forhn

In [2]:
# connect to the MongoDB
connection = pymongo.MongoClient('mongodb+srv://)

db = connection.hack     # attach to db
collection = db.hack

# Update XLS

In [3]:
df_links = pd.read_excel('nhack links for parsing.xlsx', sheet_name='Links', dtype={'link': str}) #list of proposed types and names

# Select region

In [118]:
region = 93
variant=1 #0 или 1
election_links = list(df_links[df_links['region']==region].link)
print('Total elections links:', len(election_links))

Total elections links: 6


# Select election link to parse

In [137]:
url = election_links[5].strip()
url

'http://www.crimea.vybory.izbirkom.ru/region/region/crimea?action=show&root=1&tvd=2932000293076&vrn=2932000293067&region=93&global=&sub_region=93&prver=0&pronetvd=0&vibid=2932000293067&type=220'

# First Page 

In [138]:
r = requests.get(url)
if r.status_code != requests.codes.ok:
    print('Problem to get url', r.status_code)

#parse first page for links to all
election_info = forhn.parse_first_page(r.text, variant=variant)

election={}
election['type'] = election_info['type']
election['name'] = election_info['name']
election['date_of_vote'] = election_info['date_of_vote']

#extract region 
params = url.split('?')[1].split('&')
params2 = [param.split('=') for param in params]
params3 = [[param[0],param[1]] for param in params2]
region = [param[1] for param in params3 if param[0]=='region']
election['region'] = int(region[0])
election['as_of'] = datetime.today()
election['url'] = url.strip()

print('Total persons:', election_info['total_records'], '\n')
#info for records
print(election)

#add first page 
election_info['links'].append(url.strip())

Row in info: 2
Columns: 8
Total persons: 7 

{'type': 'сведения о кандидатах, выдвинутых по одномандатным (многомандатным) избирательным округам', 'name': 'дополнительные выборы депутата государственного совета республики крым первого созыва по ленинскому одномандатному избирательному округу №12', 'date_of_vote': datetime.datetime(2017, 9, 10, 0, 0), 'region': 93, 'as_of': datetime.datetime(2019, 4, 28, 7, 2, 52, 506422), 'url': 'http://www.crimea.vybory.izbirkom.ru/region/region/crimea?action=show&root=1&tvd=2932000293076&vrn=2932000293067&region=93&global=&sub_region=93&prver=0&pronetvd=0&vibid=2932000293067&type=220'}


# Parsing

In [139]:
for link in election_info['links']:
    
    list_of_docs = []# list of records
    print(link, '\nParsing...')
    r = requests.get(link.strip())
    if r.status_code != requests.codes.ok:
        print('Problem to get url', r.status_code)
    persons = forhn.parse_page(r.text, variant)
    
    for data in persons:
        
        print(data[0])# for progress
        
        record={} #info about person
        record['election'] = election
        
        #info from page
        for i in range(0,len(election_info['columns'])):
            if i==0:
                continue
            if i==1:
                record['url'] = data[i]
                continue

            record[election_info['columns'][i]] = data[i]
    
        r = requests.get(record['url'])
        if r.status_code != requests.codes.ok:
            print('Problem to get url', r.status_code)

        person=forhn.parse_person_page(r.text, variant=variant)
        
        #info from person page 
        keys = list(person.keys())
        
        #name and sex
        member = forhn.parse_name_and_sex(person['ФИО'])
        if member:
            key_member = member.keys()
            if 'sex' in key_member:
                record['sex'] = member['sex']
            if 'name' in key_member:
                record['name'] = member['name']
        else:
            record['name_str'] = person[keys[0]]
        
        #just copy
        if  person[keys[1]]:
            record['date of birth'] = person[keys[1]]
            
        if  person[keys[2]]:
            record['place of birth'] = person[keys[2]]

        if  person[keys[3]]:
            record['location'] =  person[keys[3]]

        if person[keys[4]]:
            record['education'] =  person[keys[4]]

        if person[keys[5]]:
            record['work'] = person[keys[5]]

        if person[keys[6]]:
            record['position'] =  person[keys[6]]

        if person[keys[7]]:
            record['non-perm deputy'] =  person[keys[7]]

        if person[keys[8]]:
            record['conviction'] =  person[keys[8]]

        if person[keys[9]]:
            record['participant status'] =  person[keys[9]]
        
        list_of_docs.append(record)

        
    try:
        collection.insert_many(list_of_docs)
    except Exception as e:
        print("Error trying to insert collection:", type(e), e)
    print('Saved\n')

http://www.crimea.vybory.izbirkom.ru/region/region/crimea?action=show&root=1&tvd=2932000293076&vrn=2932000293067&region=93&global=&sub_region=93&prver=0&pronetvd=0&vibid=2932000293067&type=220 
Parsing...
1
2
3
4
5
6
7
Saved



In [13]:
election_info['links']

['http://www.mordov.vybory.izbirkom.ru/region/region/mordov?action=show&root=1&tvd=2132000677166&vrn=2132000677161&region=13&global=&sub_region=13&prver=1&pronetvd=1&vibid=2132000677161&type=220&report_mode=1']